In [55]:
import requests
from bs4 import BeautifulSoup
import re  # 본문의 태그 제거용
import pandas as pd
import time
import os
from urllib.parse import urljoin
import json

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

from selenium.webdriver.support.select import Select  # <select> 쉽게 다루기 위함

In [3]:
# !pip install selenium

In [ ]:
# 크롤링 출처
# https://fortune13.nate.com/v1/dream/dream/dreamcat.asp

In [57]:
# 기본 URL 정보
base_url = "https://fortune13.nate.com/v1/dream/dream/"

# 최종 결과를 저장할 최상위 딕셔너리
total_data = {}

# (1단계) 대분류 링크 수집
main_categories_paths = [f"dreamcat.asp?cat1={i}" for i in range(1, 9)]

for cat1_path in main_categories_paths:
    cat1_full_url = urljoin(base_url, cat1_path)
    response = requests.get(cat1_full_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    main_category_name = soup.select_one('div.searchlist2_ttl li a strong').text.strip()
    total_data[main_category_name] = {}
    print(f"--- 대분류 '{main_category_name}' 처리 중 ---")

    # (2단계) 소분류 목록 페이지 링크 수집
    subcategory_links = soup.select('div.wDream table tr td a')

    for sub_link in subcategory_links:
        sub_category_name = sub_link.text.strip()
        sub_category_list_url = urljoin(base_url, sub_link['href'])

        total_data[main_category_name][sub_category_name] = []
        print(f"  -> 소분류 '{sub_category_name}' 처리 중...")

        sub_list_response = requests.get(sub_category_list_url)
        sub_list_soup = BeautifulSoup(sub_list_response.content, 'html.parser')

        # (3단계) 최종 해몽 내용 페이지 링크를 모두 찾아, 각각 접속 후 크롤링
        # 소분류 목록 페이지에 있는 모든 최종 페이지 링크를 가져옵니다.
        final_page_links = sub_list_soup.select('div.wDream table tr td a')

        for final_link in final_page_links:
            # 최종 해몽 내용 페이지 URL
            final_page_url = urljoin(base_url, final_link['href'])

            # 페이지에 접속
            final_response = requests.get(final_page_url)
            final_soup = BeautifulSoup(final_response.text, 'html.parser')

            # 꿈, 해몽 크롤링
            try:
                view_content_area = final_soup.find('div', class_='wDream')
                dream_title = view_content_area.select_one('h4 span').text.strip()
                dream_interpretation = view_content_area.select_one('div.grayBg').text.strip()

                # 추출한 데이터를 딕셔너리에 추가
                total_data[main_category_name][sub_category_name].append({
                    '꿈': dream_title,
                    '해몽': dream_interpretation
                })
                print(f"    -> {dream_title[:20]}... 저장 완료")

            except AttributeError:
                # 간혹 구조가 다른 페이지가 있을 경우를 대비한 예외 처리
                print(f"    -> [에러] {final_page_url} 페이지의 구조가 달라 내용을 가져올 수 없습니다.")

            # 서버에 부담을 주지 않기 위해 요청 사이에 짧은 간격을 둡니다.
            time.sleep(0.1)

print("\n--- 모든 크롤링 작업 완료 ---")

# (4단계) 최종 결과를 JSON 파일로 저장
file_path = 'dream.json'
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(total_data, f, ensure_ascii=False, indent=2)

print(f"\n✅ 데이터가 '{file_path}' 파일로 저장되었습니다.")

--- 대분류 '"사람/인물"' 처리 중 ---
  -> 소분류 '가족' 처리 중...
    -> "친구나 가족이 라일락꽃을 가지고 들... 저장 완료
    -> "가족이 토끼 새끼를 안고 들어오는 ... 저장 완료
    -> "가족이 죽었는데도 감정이 전혀 흔들... 저장 완료
    -> "가족과 같이 배를 타고 있는 꿈"... 저장 완료
    -> "가족과 외출하는 꿈"... 저장 완료
    -> "가족 중의 한 사람이 돼지떼를 몰고... 저장 완료
    -> "가족과 함께 차를 몰고 야외로 떠나... 저장 완료
    -> "온 가족이 모여서 손뼉을 치고 웃는... 저장 완료
    -> "실제로 별거 중인 가족과 함께 있는... 저장 완료
    -> "헤어져있던 가족이 모두 집으로 돌아... 저장 완료
    -> "가족들과 즐겁게 지내고 있을 때 불... 저장 완료
    -> "한가족이 모여앉아 즐거워하는 꿈"... 저장 완료
    -> "한가족이 모여 잔치를 하는 꿈"... 저장 완료
    -> "가족이 다 모여있는 꿈"... 저장 완료
  -> 소분류 '부부' 처리 중...
    -> "부부가 몸에 지니고 있던 반지나 그... 저장 완료
    -> "배우자가 다른 이성과 걷거나 산책하... 저장 완료
    -> "부부가 서로 이혼하고 헤어지는 꿈"... 저장 완료
    -> "배우자의 머리를 빗겨주거나 감겨주는... 저장 완료
    -> "부부가 목청을 높여 크게 싸우는 꿈... 저장 완료
    -> "부부가 같이 앉아 있는 꿈"... 저장 완료
    -> "배우자가 아주 못생겨서 속상해하는 ... 저장 완료
    -> "부부 동반 모임에 나가는 꿈"... 저장 완료
    -> "부부가 서로 화풀이를 하는 꿈"... 저장 완료
    -> "부부가 사별하는 꿈"... 저장 완료
    -> "부부가 함께 술을 마시는 꿈"... 저장 완료
    -> "부부가 함께 사우나나 해수욕장에 들... 저장 완료
    -